# IA - Le compte est bon
First have a look at the instructions of the lab:
http://www.ai-junkie.com/ga/intro/gat3.html

In [22]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import operator
from enum import Enum
from collections import namedtuple
import random
import time

## Private part
You find here constants and private functions of the lab. No need to change something here. 

In [23]:
# Each operand or operator is described by 4 bits
CODE_LENGTH = 4

# In this example, we fix the number of operands to 5
NB_OPERANDS = 5

# The maximum number of operators is NB_OPERANDS - 1
# ex. 5 + 3 / 2 
# three operands: 5, 3, 2
# two operators: +, /
NB_OPERATORS = NB_OPERANDS - 1


CHROMOSOME_LENGTH = NB_OPERANDS * CODE_LENGTH + NB_OPERATORS * CODE_LENGTH

# We have three type of code: operands, operators and undefined symbols
class CodeType(Enum):
    OPERAND = 1
    OPERATOR = 2
    NOTHING = 3

# namedtuple("typename, field_names[...]") returns a new tuple subclass named 'typename'. 
# The new subclass is used to create tuple-like objects that have fields accessible 
# by attribute lookup as well as being indexable and iterable
Code = namedtuple("Code", ["code_type", "apply", "str"])

OPERATORS = {
    10: (operator.add, "+"),  # Standard operators as functions, see https://docs.python.org/3/library/operator.html
    11: (operator.sub, "-"),
    12: (operator.mul, "*"),
    13: (operator.truediv, "/")
}

def _parse_code(code):
    """ Convert bit string to a Code namedtuple """
    int_value = int(code, 2)
    if int_value >= 0 and int_value < 10:
        return Code(CodeType.OPERAND, lambda: int_value, str(int_value))
    elif int_value >= 10 and int_value <= 13:
        return Code(CodeType.OPERATOR, OPERATORS[int_value][0], OPERATORS[int_value][1])
    else:
        return Code(CodeType.NOTHING, None, "_")
    
def _decode(individual):
    """ Parse each code of the full chromosome (aka individual) """
    chromosome_str = "".join([str(gene) for gene in individual])
    codes = [_parse_code(chromosome_str[i: i + CODE_LENGTH]) for i in range(0, len(chromosome_str), CODE_LENGTH)]
    return codes

## Public part
You find here functions that may be interesting for you. No need to change something here. 

In [24]:
def displayable_chromosome(individual):
    """ Convert chromosome to a readable format (e.g. 3 + 5 / 6) """
    return " ".join(code.str for code in _decode(individual))

def compute_chromosome(individual):
    """ Compute operations hidden in the chromosome """
    codes = _decode(individual)
    first_operand = None
    operation = None
    snd_operand = None
    result = 0
    for code in codes:
        if not first_operand:
            if code.code_type == CodeType.OPERAND:
                first_operand = code.apply()
        elif not operation:
            if code.code_type == CodeType.OPERATOR:
                operation = code.apply
        elif not snd_operand:
            if code.code_type == CodeType.OPERAND:
                snd_operand = code.apply()
                try:
                    result = operation(first_operand, snd_operand)
                except ZeroDivisionError:
                    pass
                first_operand = result
                operation = None
                snd_operand = None
    return result

## Deap framework
You find here a preparation of tools necessary for our algorithm. No need to change something here.

In [25]:
toolbox = base.Toolbox()

The **fitness** will measure the proximity between the result of the chromosome and a target value.
The lower the proximity is the better is our chromosome so we are in a minimization problem. 

The abstract type `base.Fitness` has attributes:
- `values` representing fitness  
- `weights` used to define a maximization/minimization by setting 1.0 or -1.0.

Be aware that `values` and `weights` must be tuples.

In [26]:
def fitness(individual, target):
    return (abs(compute_chromosome(individual) - target),) # Tuple !
toolbox.register("fitness", fitness)

In [27]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

A `deep.creator.Individual` will be list of gene with an attribute fitness of type `deep.creator.FitnessMin` just created.

In [28]:
creator.create("Individual", list, fitness=creator.FitnessMin) 

- **Crossover** between two individuals will be a simple one point crossover.
- **Mutation** of a individual will flip the bit in the gene with a probability of 10%.
- **Selection** of *k* individuals will be done using a tournament.

In [29]:
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
# toolbox.register("select", tools.selRoulette) # cannot be used for a minimization problem
toolbox.register("select", tools.selTournament) # need an additional parameter turnsize
# toolbox.register("select", tools.selBest)

Finally we provide initiations operations. A **population** will be a list of `deep.creator.Individual`. Each gene will be set randomly to 0 or 1.

In [30]:
toolbox.register("init_gene", random.randint, 0, 1)
toolbox.register("init_individual", tools.initRepeat, creator.Individual, toolbox.init_gene, CHROMOSOME_LENGTH)
toolbox.register("init_population", tools.initRepeat, list, toolbox.init_individual)

## Solve "Le compte est bon"
It'y sour turn! Using Deap and previous tools design a loop to obtain **TARGET** value in a maximum time of **MAX_TIME**. Break the loop if an individual is optimal before **MAX_TIME** (i.e. his fitness = 0).

In [31]:
TARGET = 126
MAX_TIME = 5  # seconds

Some hints:
- Take a look at Deap documentation https://deap.readthedocs.io/en/master/index.html
- Create a small population `init_population(n)` and use `compute_chromosome(individual)`, `display_chromosome(individual)` and `individual.fitness` to clearly understand what is an Individual.

Good luck!

In [32]:
def evaluate_population(population, target):
    for ind in population:
        ind.fitness.values = toolbox.fitness(ind, target)

In [33]:
def evaluate_population_2(population, target):
    fitnesses = [toolbox.fitness(ind, target) for ind in population]
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

In [34]:
def find_winners(population):
    winners = [ind for ind in population if ind.fitness.values[0] == 0]
    return winners

In [35]:
toolbox.register("evaluate", evaluate_population)

In [36]:
start_time = inter_time = time.time()
population = toolbox.init_population(n=160)
tournsize = 8
# crossover probability
CXPB = 0.7

# mutation probability (of a chromosome)
# compare with the indpb=0.1 in the previous line toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
MUTPB = 0.2

toolbox.evaluate(population, TARGET)
solution = None

In [37]:
number_of_iterations = 0
while len(find_winners(population)) == 0 and inter_time - start_time < MAX_TIME:
    children = toolbox.select(population, len(population), tournsize)
         
    # Full alternative ---
    # Clone the selected individuals, 
    # TODO: test your code without the cloning, print the 
    children = list(map(toolbox.clone, children))
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(children[::2], children[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)            

    for mutant in children:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            
    # ---
    
    # Shorcut alternative ---
    #children = algorithms.varAnd(offspring, toolbox, 0.7, 0.2)
    # ---
    
    evaluate_population(children, TARGET)
    population = children
    
    # Search for the solution
    fitnesses = [ind.fitness.values[0] for ind in population]
    min_fit = min(fitnesses)
    best = population[fitnesses.index(min_fit)]
    if not solution or best.fitness.values[0] < solution.fitness.values[0]:
        solution = best
        
    inter_time = time.time()
    
    number_of_iterations += 1

duration = time.time() - start_time

# TODO 4 :) In which case the following line is execuded?
if not solution:
    solution = find_winners(population)[0]

elif duration >= MAX_TIME:
    print('Timeout - solution not found')

else:
    print('Solution found!')

Solution found!


In [38]:
compute_chromosome(solution)

126

In [39]:
solution.fitness

deap.creator.FitnessMin((0.0,))

In [40]:
print(displayable_chromosome(solution))

6 * * 7 * 3 7 9 2


In [41]:
#print the total time and the number iterations
print(f'Duration: {duration}')
print(f'Number of iterations: {number_of_iterations}')

Duration: 0.052857160568237305
Number of iterations: 1


In [42]:
# Display the final population
def printMatrix(matrix):
    for i, row in enumerate(matrix):
        print(i, ''.join(str(row)))
        
printMatrix(population)

0 [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1]
1 [0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]
2 [0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
3 [0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
4 [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
5 [0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0]
6 [0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
7 [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]
8 [0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]
9